In [9]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import unidecode


In [10]:
import geopandas as gpd

In [11]:
import numpy as np
from shapely.geometry import Point, Polygon

import math



# Grid Shapes

Para crear un grid con polígonos geoespaciales y asignar a cada punto según su latitud y longitud a un polígono correspondiente, se puede utilizar la biblioteca geopandas en Python, que extiende las capacidades de pandas para trabajar con datos espaciales. También usaremos shapely para manipular y analizar datos geométricos planos, y numpy para manejar operaciones numéricas.

Primero, generamos un conjunto de polígonos que representen cada celda del grid sobre el área de interés (en este caso, México). Cada polígono puede ser identificado de manera única, por ejemplo, usando una convención de nomenclatura como PG1, PG2, etc.

### Tamaños del grid

    - Dimensiones del grid: 120, 10, -82, 37 (son las puntos cardinales que cubren el espacio donde se encuentra Mexico)
    
    - Tamaño de los poligonos que integran el grid:
    
CRS = EPSG:4326 para las coordenadas geográficas

Metros = epsg=6366 (10000) = 10km ¿Por que?

Usando "EPSG:4326", sacamos una aproximacion en metros para el tamaño de nuestros grids, directamente relacionada con las zonas a analizar.



In [14]:
def longitud_por_porcentaje_lat(latitud, porcentaje):
    """
    Calcula la longitud en metros de un porcentaje de 1 grado de longitud a una latitud dada.
    
    Args:
    latitud (float): La latitud en grados.
    porcentaje (float): El porcentaje de 1 grado de longitud. Por ejemplo, 0.5 para 50%, 0.6 para 60%.
    
    Returns:
    float: La longitud en metros del porcentaje dado de 1 grado de longitud en la latitud especificada.
    """
    # Convertir latitud a radianes
    lat_rad = math.radians(latitud)
    # Calcular longitud en km por grado de longitud
    longitud_km = math.cos(lat_rad) * 111.32
    # Convertir a metros y aplicar el porcentaje
    return longitud_km * 1000 * porcentaje

# Ejemplo de uso: Calcular el % de 1 grado de longitud en la latitud 10°
longitud_10_50porc = longitud_por_porcentaje_lat(10, .7)
# Ejemplo de uso: Calcular el % de 1 grado de longitud en la latitud 37°
longitud_37_50porc = longitud_por_porcentaje_lat(37, .7)

print(f"% de 1 grado de longitud a 10° latitud: {longitud_10_50porc:.2f} metros")
print(f"% de 1 grado de longitud a 37° latitud: {longitud_37_50porc:.2f} metros")


% de 1 grado de longitud a 10° latitud: 76740.16 metros
% de 1 grado de longitud a 37° latitud: 62232.87 metros


In [17]:
def longitud_por_porcentaje_latitud(porcentaje):
    """
    Calcula la longitud en metros de un porcentaje de 1 grado de latitud.
    
    Args:
    porcentaje (float): El porcentaje de 1 grado de latitud. Por ejemplo, 0.5 para 50%, 0.6 para 60%.
    
    Returns:
    float: La longitud en metros del porcentaje dado de 1 grado de latitud.
    """
    longitud_km_por_grado = 111  # Aproximadamente 111 km por grado de latitud
    return longitud_km_por_grado * 1000 * porcentaje  # Convertir a metros y aplicar el porcentaje

# Ejemplo de uso: Calcular el 50% de 1 grado de latitud
longitud_50porc_latitud = longitud_por_porcentaje_latitud(0.7)
# Ejemplo de uso: Calcular el 60% de 1 grado de latitud
longitud_60porc_latitud = longitud_por_porcentaje_latitud(0.7)

print(f"50% de 1 grado de latitud: {longitud_50porc_latitud:.2f} metros")
print(f"60% de 1 grado de latitud: {longitud_60porc_latitud:.2f} metros")


50% de 1 grado de latitud: 77700.00 metros
60% de 1 grado de latitud: 77700.00 metros


In [19]:
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []
    ids = []

    id_contador = 1  # Iniciar contador para los ID de polígonos
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
            ids.append(f"PG{id_contador}")  # Asignar un ID único a cada polígono
            id_contador += 1

    grid = gpd.GeoDataFrame({'ID_Poligono': ids, 'geometry': poligonos})
    return grid

# Generar el grid con los límites aproximados de México y celdas de 1 grado por 1 grado

xmin, ymin, xmax, ymax = -120, 10, -82, 37

grid_1 = crear_grid(xmin, xmax, ymin, ymax, 1, 1)
grid_punto7 = crear_grid(xmin, xmax, ymin, ymax, .7, .7)

#Ahora, 'grid' es un GeoDataFrame que contiene el 'ID_Poligono' y su 'geometry'
print(grid_1.head())
print(grid_punto7.head())

  ID_Poligono                                           geometry
0         PG1  POLYGON ((-120.00000 10.00000, -119.00000 10.0...
1         PG2  POLYGON ((-120.00000 11.00000, -119.00000 11.0...
2         PG3  POLYGON ((-120.00000 12.00000, -119.00000 12.0...
3         PG4  POLYGON ((-120.00000 13.00000, -119.00000 13.0...
4         PG5  POLYGON ((-120.00000 14.00000, -119.00000 14.0...
  ID_Poligono                                           geometry
0         PG1  POLYGON ((-120.00000 10.00000, -119.30000 10.0...
1         PG2  POLYGON ((-120.00000 10.70000, -119.30000 10.7...
2         PG3  POLYGON ((-120.00000 11.40000, -119.30000 11.4...
3         PG4  POLYGON ((-120.00000 12.10000, -119.30000 12.1...
4         PG5  POLYGON ((-120.00000 12.80000, -119.30000 12.8...


In [20]:
# Guardar grid_1 como Shapefile
grid_1.to_file("/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid_1x1.shp")

# Guardar grid_punto7 como Shapefile
grid_punto7.to_file("/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid_punto7x7.shp")


/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_59709/967252687.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_1.to_file("/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid_1x1.shp")
/var/folders/68/6zkv8z4j4rl6ds90br2rb7ph0000gn/T/ipykernel_59709/967252687.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  grid_punto7.to_file("/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/1. Mapas/2. arcgis(general)/grids/grid_punto7x7.shp")


In [ ]:
# Definir las listas de estados para cada ruta, asegurándose de que todas las entradas están correctamente formateadas como strings
estados_ruta_golfo = ['Tabasco', 'Nuevo Leon', 'Veracruz', 'Tamaulipas']
estados_ruta_pacifico = ['Guerrero', 'Michoacan', 'Jalisco', 'Nayarit', 'Sinaloa', 'Sonora', 'Baja California', 'Baja California Sur','Colima']
estados_ruta_centro = ['San Luis Potosi','Estado de Mexico', 'Tlaxcala', 'Ciudad de Mexico', 'Puebla', 'Morelos', 'Hidalgo', 'Zacatecas', 'Durango', 'Chihuahua', 'Coahuila', 'Queretaro', 'Aguascalientes', 'Guanajuato']
ruta_sur = ['Chiapas', 'Oaxaca', 'Tabasco', 'Campeche', 'Quintana Roo', 'Yucatan']

# Actualizar la función para asignar la ruta basada en el estado
#def asignar_ruta(estado):
#    estado = estado.strip()  # Eliminar espacios en blanco al principio y al final
#    if estado in estados_ruta_golfo:
#        return 'Ruta del Golfo'
#    elif estado in estados_ruta_pacifico:
#        return 'Ruta del Pacifico'
#    elif estado in estados_ruta_centro:
#        return 'Ruta Centro'
#    elif estado in ruta_sur:
#        return 'Ruta Sur'
#    else:
#        return 'Ruta no identificada'  # Para estados que no coinciden con las listas

# Aplicar la función al DataFrame para crear una nueva columna 'Ruta'
#datos_csv['ruta'] = datos_csv['estado'].apply(asignar_ruta)

In [ ]:
# Reordenar las columnas para colocar 'Estado' después de 'Ciudad'
#column_order = ['ciudad', 'estado', 'ruta','detenciones']

# Aplicar el nuevo orden de columnas al DataFrame
#datos_csv = datos_csv[column_order]

In [ ]:
# Obtenemos los valores únicos de la columna 'ruta'
#valores_unicos_ruta = datos_csv['ruta'].unique()

# Imprimimos los valores únicos
#print(valores_unicos_ruta)

In [ ]:
# Initialize the geolocator using OSM Nominatim
#geolocator = Photon(user_agent="measurement", timeout=10)

# Define a function to get latitude and longitude
#def get_lat_lon(address):
#    try:
#        location = geolocator.geocode(address)
#        if location:
#            return location.latitude, location.longitude
#        else:
#            return '', ''  # Return empty strings if location is not found
#    except AttributeError:
#        return '', ''

# Apply the function to the 'City' column and split the results into 'Latitude' and 'Longitude' columns
#datos_csv[['latitud', 'longitud']] = datos_csv['ciudad'].apply(lambda x: pd.Series(get_lat_lon(x)))

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
#ruta_archivo_csv = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
#datos_csv.to_csv(ruta_archivo_csv, index=False)

#print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv}")


In [ ]:
# Cargar el archivo CSV
#datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [ ]:
# Convertir la columna 'detenciones' a entero
#datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Identificar los índices de la primera fila de cada categoría en 'estado'
#indices_a_eliminar = datos_csv1.groupby('estado').head(1).index

# Eliminar esas filas del DataFrame
#datos_csv1 = datos_csv1.drop(indices_a_eliminar)




In [ ]:
# Verificar los cambios
#print(datos_csv1.head())


In [ ]:
# Agregar la columna 'tipo' con la etiqueta 'Presencia Migración Irregular' para todas las filas
#datos_csv1['tipo'] = 'Presencia Migración Irregular'

# Verificar los cambios mostrando las primeras filas del DataFrame
#print(datos_csv1.head())


In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
#ruta_archivo_csv1 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
#datos_csv1.to_csv(ruta_archivo_csv1, index=False)

#print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv1}")


### Grid Shape

Para crear un grid con polígonos geoespaciales y asignar a cada punto según su latitud y longitud a un polígono correspondiente, se puede utilizar la biblioteca geopandas en Python, que extiende las capacidades de pandas para trabajar con datos espaciales. También usaremos shapely para manipular y analizar datos geométricos planos, y numpy para manejar operaciones numéricas.



Crear el Grid de Polígonos: Primero, generamos un conjunto de polígonos que representen cada celda del grid sobre el área de interés (en este caso, México). Cada polígono puede ser identificado de manera única, por ejemplo, usando una convención de nomenclatura como PG1, PG2, etc.

In [ ]:
def crear_grid(xmin, xmax, ymin, ymax, ancho_celda, alto_celda):
    cols = list(np.arange(xmin, xmax, ancho_celda))
    rows = list(np.arange(ymin, ymax, alto_celda))
    poligonos = []
    ids = []

    id_contador = 1  # Iniciar contador para los ID de polígonos
    for x in cols:
        for y in rows:
            poligonos.append(Polygon([(x, y), (x+ancho_celda, y), (x+ancho_celda, y-alto_celda), (x, y-alto_celda)]))
            ids.append(f"PG{id_contador}")  # Asignar un ID único a cada polígono
            id_contador += 1

    grid = gpd.GeoDataFrame({'ID_Poligono': ids, 'geometry': poligonos})
    return grid

# Generar el grid con los límites aproximados de México y celdas de 1 grado por 1 grado
xmin, ymin, xmax, ymax = -120, 10, -82, 37
grid = crear_grid(xmin, xmax, ymin, ymax, 1, 1)

 Ahora, 'grid' es un GeoDataFrame que contiene el 'ID_Poligono' y su 'geometry'
#print(grid.head())

In [ ]:
# Cargar el archivo CSV
#datos_csv1 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [ ]:
# Actualizar Campeche, Campeche
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'latitud'] = 19.830285751988043
datos_csv1.loc[(datos_csv1['ciudad'] == 'Campeche, Campeche'), 'longitud'] = -90.53551463998544

# Actualizar Colima, Colima
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'latitud'] = 19.24487815537049
datos_csv1.loc[(datos_csv1['ciudad'] == 'Colima, Colima'), 'longitud'] = -103.7250306766154

# Actualizar Puebla, Puebla
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'latitud'] = 19.038406227976886
datos_csv1.loc[(datos_csv1['ciudad'] == 'Puebla, Puebla'), 'longitud'] = -98.2069491554557


datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'latitud'] = 19.039018782328817
datos_csv1.loc[(datos_csv1['ciudad'] == 'Zacatecas, Zacatecas'), 'longitud'] = -98.20248616428334

In [ ]:
datos_csv1.head(30)

In [ ]:
# Eliminar las columnas del DataFrame
#datos_csv1 = datos_csv1.drop(columns=['ID_Poligono','geometry', 'codigo_ruta', 'contador_xy', 'codigo_unico'])

# Verificación: Muestra las primeras filas para asegurarse de que las columnas han sido eliminadas
#print(datos_csv1.head())


In [ ]:
# Convertir la columna 'detenciones' a entero
#datos_csv1['detenciones'] = pd.to_numeric(datos_csv1['detenciones'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Paso 2: Asignar Puntos a Polígonos
puntos = [Point(xy) for xy in zip(datos_csv1.longitud, datos_csv1.latitud)]
geo_df = gpd.GeoDataFrame(datos_csv1, geometry=puntos)

# Realizar la asignación
geo_df = gpd.sjoin(geo_df, grid, how='left', op='within')

# Paso 3: Actualizar DataFrame
datos_csv1['ID_Poligono'] = geo_df['ID_Poligono']

In [ ]:
# Asegurándose de que datos_csv2 es un GeoDataFrame
if not isinstance(datos_csv1, gpd.GeoDataFrame):
    # Si datos_csv2 ya tiene una columna 'geometry', úsala; de lo contrario, se necesita definir una
    if 'geometry' in datos_csv1.columns:
        datos_csv1 = gpd.GeoDataFrame(datos_csv1, geometry='geometry')
    else:
        # Aquí se asume que no hay una columna 'geometry' existente y que quizás no necesites definir una en este momento
        datos_csv1 = gpd.GeoDataFrame(datos_csv1)

# Realizar el merge (unión) entre datos_csv2 y grid usando 'ID_Poligono' como clave
datos_csv1 = datos_csv1.merge(grid[['ID_Poligono', 'geometry']], on='ID_Poligono', how='left')


In [ ]:
datos_csv1.tail(20)

Asignar Puntos a Polígonos: Luego, para cada punto dado por sus coordenadas de latitud y longitud en datos_csv1, determinamos en qué polígono del grid se encuentra y asignamos el identificador de ese polígono al punto.


el codigo de un punto se miraria algo asi: S-PG#-1

Para asignar un código que combine el código de ruta por estado, el identificador del polígono al que pertenece cada punto y un número ascendente basado en el orden de las coordenadas XY dentro de cada polígono

por estado, si:
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
    
por ID_Poligono: el poligono a que pertenece

y por orden asendientes de su coordenada xy dentro de cada poligono

In [ ]:
# Primero, mapeamos la columna 'ruta' a su código correspondiente
ruta_to_code = {
    'Ruta Sur': 'S',
    'Ruta del Pacifico': 'P',
    'Ruta Centro': 'C',
    'Ruta del Golfo': 'G'
}
datos_csv1['codigo_ruta'] = datos_csv1['ruta'].map(ruta_to_code)

# Asumiendo que 'datos_csv2' ya tiene una columna 'ID_Poligono',
# vamos a crear una columna que identifique el orden XY dentro de cada polígono
# Para esto, primero ordenamos el DataFrame
datos_csv1.sort_values(by=['ID_Poligono', 'latitud', 'longitud'], ascending=[True, True, True], inplace=True)

# Creamos una nueva columna que será el contador dentro de cada grupo de 'ID_Poligono'
datos_csv1['contador_xy'] = datos_csv1.groupby('ID_Poligono').cumcount() + 1

# Ahora, combinamos todo para crear el código único
datos_csv1['codigo_unico'] = datos_csv1['codigo_ruta'] + '-' + datos_csv1['ID_Poligono'] + '-' + datos_csv1['contador_xy'].astype(str)

# Revisamos el resultado
print(datos_csv1[['ciudad', 'estado', 'ruta', 'ID_Poligono', 'codigo_unico']])


Actualizar DataFrame: Finalmente, agregamos una nueva columna al DataFrame datos_csv1 para almacenar el identificador del polígono del grid al que pertenece cada punto.

In [ ]:
# Obtener el conteo de los valores únicos en la columna 'ID_Poligono'
conteo_id_poligono = datos_csv1['ID_Poligono'].value_counts()

# Mostrar el conteo
print(conteo_id_poligono)



In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv4 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/grid.csv'

# Guardar el DataFrame en el archivo CSV especificado
grid.to_csv(ruta_archivo_csv4, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv3}")

In [ ]:
grid

In [ ]:
datos_csv2

### Arreglamos detenciones

reordenar mi df datos_csv2, usando su columna estado, sort en orden alfabetico y resetear el index

In [ ]:
# Cargar el archivo CSV
datos_csv2 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


In [ ]:
# Ordenamos el DataFrame por la columna 'estado' en orden alfabético
datos_csv2_ordenado = datos_csv2.sort_values(by='estado')

# Reseteamos el índice del DataFrame ordenado
datos_csv2_ordenado.reset_index(drop=True, inplace=True)



In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv5 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv2_ordenado.to_csv(ruta_archivo_csv5, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv5}")

In [ ]:
# Cargar el archivo CSV
datos_csv3 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv')


#### AGREGamos columna con los datos de detenciones, procesamos y unimos

In [ ]:
# Cargar el archivo CSV
datos_detenciones_2023 = pd.read_csv('/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones.csv')


In [ ]:
datos_detenciones_2023 = datos_detenciones_2023.dropna()

In [ ]:
datos_detenciones_2023.head()

In [ ]:
# Cambiamos los nombres de las columnas usando el método 'rename'
datos_detenciones_2023 = datos_detenciones_2023.rename(columns={'Unnamed: 0': 'ciudad', 'Unnamed: 1': 'detenciones'})



In [ ]:
# Función para eliminar acentos en una cadena de texto, manejando valores no-cadena
def quitar_acentos(texto):
    # Si el valor es una cadena, aplicamos unidecode
    if isinstance(texto, str):
        return unidecode.unidecode(texto)
    # Si no es una cadena, lo dejamos tal cual (esto incluye NaN y números)
    return texto

# Aplicamos la función a todas las columnas de tipo object en el DataFrame
for columna in datos_detenciones_2023.select_dtypes(include=['object']).columns:
    datos_detenciones_2023[columna] = datos_detenciones_2023[columna].apply(quitar_acentos)



In [ ]:
# Función para detectar si un nombre de ciudad tiene caracteres especiales (no es alfanumérico)
def tiene_caracteres_especiales(cadena):
    return not cadena.replace(" ", "").isalnum()

# Crea una nueva columna 'estado' inicialmente llena con NaN
datos_detenciones_2023['estado'] = pd.NA

# Inicializamos una variable para almacenar el estado actual mientras iteramos
estado_actual = None

# Iteramos sobre las filas del DataFrame
for index, row in datos_detenciones_2023.iterrows():
    # Si la ciudad no tiene caracteres especiales, actualizamos estado_actual
    if not tiene_caracteres_especiales(row['ciudad']):
        estado_actual = row['ciudad']
    # Asignamos el estado actual a la columna 'estado'
    datos_detenciones_2023.at[index, 'estado'] = estado_actual

In [ ]:
# Filtramos el DataFrame para mantener solo las filas donde la columna 'ciudad' tiene caracteres especiales
datos_detenciones_2023 = datos_detenciones_2023[datos_detenciones_2023['ciudad'].apply(tiene_caracteres_especiales)]


In [ ]:
# Asegúrate de que 'datos_detenciones_2023' es el DataFrame actual después de eliminar las filas con NaN.
datos_detenciones_2023 = datos_detenciones_2023[['ciudad', 'estado', 'detenciones']]


In [ ]:
# Reemplazamos el contenido de la columna 'ciudad' después de la primera coma por el valor de la columna 'estado'
datos_detenciones_2023['ciudad'] = datos_detenciones_2023.apply(lambda fila: fila['ciudad'].split(',', 1)[0] + ", " + fila['estado'], axis=1)


In [ ]:
# Verificamos el cambio
print(datos_detenciones_2023.head(20))

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv6 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_2023.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_detenciones_2023.to_csv(ruta_archivo_csv6, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv6}")

In [ ]:
# Realizamos un merge entre datos_csv3 y datos_detenciones_2023 basándonos en 'ciudad'
datos_combinados = pd.merge(datos_csv3, datos_detenciones_2023[['ciudad', 'detenciones']],
                            on='ciudad', how='left', suffixes=('', '_nueva'))


In [ ]:
# Actualizamos la columna 'detenciones' en datos_csv3 con los valores de 'detenciones_nueva'
datos_csv3['detenciones'] = datos_combinados['detenciones_nueva']


In [ ]:
# Verificamos el cambio
print(datos_csv3.head())

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv7 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_1.csv'

# Guardar el DataFrame en el archivo CSV especificado
datos_csv3.to_csv(ruta_archivo_csv7, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv7}")

### Hacemos segundo df con geometry_range

In [ ]:
datos_csv3.head()

In [ ]:
# Seleccionamos las columnas específicas de datos_csv3
datos_csv4 = datos_csv3[['ciudad', 'estado', 'ruta', 'detenciones', 'latitud', 'longitud', 'tipo', 'ID_Poligono', 'codigo_unico']].copy()

# Reseteamos el índice del nuevo DataFrame
datos_csv3.reset_index(drop=True, inplace=True)

# Verificamos el resultado
print(datos_csv4.head())


In [ ]:
# Asegurándonos de trabajar con el DataFrame correcto, df1
gdf = gpd.GeoDataFrame(datos_csv4, geometry=gpd.points_from_xy(datos_csv4.longitud, datos_csv4.latitud))

# Estableciendo el CRS original a EPSG:4326 para las coordenadas geográficas
gdf.crs = "EPSG:4326"

# Cambiando el CRS a uno que use metros para el cálculo del buffer. Reemplazando el EPSG con un código correcto.
# Nota: El EPSG 6366 parece ser un error. Asegúrate de usar el código EPSG de UTM adecuado para tu zona.
gdf_metros = gdf.to_crs(epsg=6366)  # Reemplaza YOUR_UTM_EPSG_CODE con el código EPSG adecuado

# Aplicando el buffer de 10km para crear una nueva columna 'geometry_range' con los polígonos
gdf_metros['geometry_range'] = gdf_metros.geometry.buffer(10000)

# (Opcional) Si deseas convertir el GeoDataFrame de regreso a lat/long (EPSG:4326) para alguna otra operación,
# y quieres que la columna 'geometry_range' sea la geometría activa,
# debes hacerlo antes de cambiar el CRS de regreso a EPSG:4326.
# gdf_metros.set_geometry('geometry_range', inplace=True)

# Cambiando de regreso a EPSG:4326
gdf_final = gdf_metros.to_crs(epsg=4326)

In [ ]:
gdf_final

In [ ]:
# Especifica la ruta donde deseas guardar el archivo CSV
ruta_archivo_csv7 = '/Users/pablouriarte/Documents/1. Expediente Tec de Monterrey/1.Tesis/Mapa_Migracion_Irregular_Mexico/3. mapas/1. Mapas/Metodo_1/ciudades_rutas/ciudades detenciones/ciudades_detenciones_geometry_range.csv'

# Guardar el DataFrame en el archivo CSV especificado
gdf_final.to_csv(ruta_archivo_csv6, index=False)

print(f"El DataFrame ha sido guardado exitosamente en: {ruta_archivo_csv7}")